In [1]:
import numpy as np

import random
import matplotlib.pyplot as plt
import time

from utils.MCMC import MCMC_Simulator
from utils import RandomSeedResetter
RandomSeedResetter.reset_random_seed(0)

Using TensorFlow backend.


In [2]:
_input_train_shuffle = np.load('data/esn_lorenz/log_esn_train_shuffle.npy')
output_train_shuffle = np.load('data/lorenz/output_train_shuffle.npy')
_input_valid = np.load('data/esn_lorenz/log_esn_valid.npy')
output_valid = np.load('data/lorenz/output_valid.npy')
_input_test = np.load('data/esn_lorenz/log_esn_test.npy')
output_test = np.load('data/lorenz/output_test.npy')

DIM_OUTPUT = len(output_train_shuffle[0, :])
SIZE_LEAK = len(_input_train_shuffle[:])
SIZE_DEEP_ESN = len(_input_train_shuffle[0, 0, :])
SIZE_VARIABLE = SIZE_LEAK*SIZE_DEEP_ESN

SIZE_TRAIN_SHUFFLE = len(_input_train_shuffle[0, :])
SIZE_VALID = len(_input_valid[0, :])
SIZE_TEST = len(_input_test[0, :])

input_train_shuffle = np.zeros((SIZE_TRAIN_SHUFFLE, SIZE_VARIABLE))
input_valid = np.zeros((SIZE_VALID, SIZE_VARIABLE))
input_test = np.zeros((SIZE_VALID, SIZE_VARIABLE))
for idx in range(SIZE_LEAK):
    input_train_shuffle[:, idx*SIZE_DEEP_ESN:(idx+1)*SIZE_DEEP_ESN] = _input_train_shuffle[idx, :]
    input_valid[:, idx*SIZE_DEEP_ESN:(idx+1)*SIZE_DEEP_ESN] = _input_valid[idx, :, :]
    input_test[:, idx*SIZE_DEEP_ESN:(idx+1)*SIZE_DEEP_ESN] = _input_test[idx, :]

In [3]:
mcmc = []
SIZE_TEMP = 10
SIZE_DATA = 10000
SIZE_SIMULATION = 100

temp_inv = np.zeros((SIZE_TEMP))
for idx_temp in range(SIZE_TEMP):
    temp_inv[idx_temp] = 2**(idx_temp-SIZE_TEMP)
    obj_mcmc = MCMC_Simulator(
                            type_IC = 'BIC',
                            temp_inv = temp_inv[idx_temp],
                            SIZE_SIMULATION=SIZE_SIMULATION,
                            SIZE_DATA = SIZE_DATA,
                            SIZE_VARIABLE=SIZE_VARIABLE)
    mcmc.append(obj_mcmc)

In [4]:
state_select_best = np.zeros((SIZE_SIMULATION, SIZE_VARIABLE))
state_IC_temp = np.zeros((SIZE_SIMULATION, SIZE_TEMP))
state_IC_temp_best = np.zeros((SIZE_SIMULATION, SIZE_TEMP))
state_IC_best = np.zeros((SIZE_SIMULATION))
IC_best = 1000000

input_train_sim = np.zeros((SIZE_TRAIN_SHUFFLE, SIZE_VARIABLE))
output_train_sim = np.zeros((SIZE_TRAIN_SHUFFLE, DIM_OUTPUT))

time_init = time.time()
for idx_sim in range(SIZE_SIMULATION):
    # shuffle the dataset
    idx_rand = random.sample(range(SIZE_TRAIN_SHUFFLE), k=SIZE_TRAIN_SHUFFLE)
    for idx_batch in range(SIZE_TRAIN_SHUFFLE):
        idx_rand = idx_rand[idx_batch]
        input_train_sim[idx_batch, :] = input_train_shuffle[idx_rand, :]
        output_train_sim[idx_batch, :] = output_train_shuffle[idx_rand, :]
    
    for idx_temp in range(SIZE_TEMP):
        mcmc[idx_temp].simulateMCMC(
            input_train=input_train_sim[:SIZE_DATA, :],
            output_train=output_train_sim[:SIZE_DATA, :],
            input_valid=input_valid[:, :],
            output_valid=output_valid[:, :])
    
    idx_exchange = int(np.random.randint(0, SIZE_TEMP-1, 1))
    IC_pre = MCMC_Simulator.calculateIC(
        mcmc[0].type_IC,
        input_train=input_train_sim[:SIZE_DATA, :],
        output_train=output_train_sim[:SIZE_DATA, :],
        input_valid=input_valid[:, :],
        output_valid=output_valid[:, :],
        state_select=mcmc[idx_exchange].state_select)
    IC_post = MCMC_Simulator.calculateIC(
        mcmc[0].type_IC,
        input_train=input_train_sim[:SIZE_DATA, :],
        output_train=output_train_sim[:SIZE_DATA, :],
        input_valid=input_valid[:, :],
        output_valid=output_valid[:, :],
        state_select=mcmc[idx_exchange+1].state_select)
    
    prob_change = np.exp( \
        ((temp_inv[idx_exchange+1]-temp_inv[idx_exchange]) *  (IC_post-IC_pre)).clip(-100, 100))
    if np.random.uniform(0, 1) < prob_change:
        state_select_pre = mcmc[idx_exchange].state_select
        state_select_post = mcmc[idx_exchange+1].state_select
        mcmc[idx_exchange].state_select = state_select_post
        mcmc[idx_exchange+1].state_select = state_select_pre
        mcmc[idx_exchange].IC = IC_post
        mcmc[idx_exchange+1].IC = IC_pre
        if IC_post < mcmc[idx_exchange].IC_best:
            mcmc[idx_exchange].state_select_best = state_select_post
            mcmc[idx_exchange].IC_best = IC_post
        if IC_pre < mcmc[idx_exchange+1].IC_best:
            mcmc[idx_exchange+1].state_select_best = state_select_pre
            mcmc[idx_exchange+1].IC_best = IC_pre
    
    for idx_temp in range(SIZE_TEMP):
        state_IC_temp[idx_sim, idx_temp] = mcmc[idx_temp].IC
        state_IC_temp_best[idx_sim, idx_temp] = mcmc[idx_temp].IC_best
        if mcmc[idx_temp].IC_best < IC_best:
            IC_best = mcmc[idx_temp].IC_best
            select_best = mcmc[idx_temp].state_select_best[:]
    
    state_IC_best[idx_sim] = IC_best
    state_select_best[idx_sim, :] = select_best[:]
    
    time_end = time.time()
    print('idx_sim:', idx_sim, ', IC_best:', '{:.2f}'.format(IC_best_all), \
              ', size_select:', len(np.nonzero(state_select_bestl[:])[0]))
    print('The index of the simnulation:', int((time_end-time_init)/60), 'min')

KeyboardInterrupt: 

In [ ]:
for idx_temp in range(SIZE_TEMP):
    plt.plot(np.log(state_IC_temp[:, idx_temp]), label=idx_temp)
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim([0, SIZE_SIMULATION-1])
plt.show

In [ ]:
for idx_temp in range(SIZE_TEMP):
    plt.plot(np.log(state_IC_temp_best[:, idx_temp]), label=idx_temp)
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim([0, SIZE_SIMULATION-1])
plt.show

In [ ]:
plt.plot(np.log(state_IC_best[:]), label='All')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim([0, SIZE_SIMULATION-1])
plt.show

In [ ]:
print(len(np.nonzero(state_select_best[:])[0]))

In [ ]:
freq_leak = np.zeros((SIZE_LEAK))
for idx_leak in range(SIZE_LEAK):
    freq_leak[idx_leak] = np.sum(state_select_best[idx_leak*1000:(idx_leak+1)*1000])
plt.plot(freq_leak[:], label='best')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim([0, 4])
plt.show

In [ ]:
SIZE_DEPTH = 10
freq_depth = np.zeros((SIZE_DEPTH))
for idx_leak in range(SIZE_LEAK):
    for idx_depth in range(SIZE_DEPTH):
        freq_depth[idx_depth] += np.sum(
            state_select_best_all[idx_depth*100+idx_leak*1000 \
            :(idx_depth+1)*100+idx_leak*1000])
plt.plot(freq_depth[:], label='best')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim([0, 9])
plt.show

In [ ]:
input_train_select = np.zeros((SIZE_TRAIN_SHUFFLE, SIZE_VARIABLE))
input_train_select[:, :] = input_train_shuffle[:, :] * state_select_best[:]

input_train_select_const = np.zeros((SIZE_TRAIN_SHUFFLE, SIZE_VARIABLE+1))
input_train_select_const[:, :-1] =  input_train_select[:, :]
input_train_select_const[:, -1] = np.ones((SIZE_TRAIN_SHUFFLE))

pinv_input_train_select = np.linalg.pinv(input_train_select_const[:, :])
coeff_train_select = pinv_input_train_select[:, :] @ output_train_shuffle[:, :]
output_train_select_pred = input_train_select_const[:, :]@coeff_train_select[:, :]

input_test_select_const = np.zeros((SIZE_TEST, SIZE_VARIABLE+1))
input_test_select_const[:, :-1] =  input_test[:, :] * state_select_best_all[:]
input_test_select_const[:, -1] = np.ones((SIZE_TEST))

output_test_select = input_test_select_const[:, :] @ coeff_train_select[:, :]
error_test_select = output_test[:, :] - output_test_select[:, :]
mse_test_select = np.average(error_test_select[:, :]**2, axis=0)

In [ ]:
input_train_all_const = np.zeros((SIZE_TRAIN_SHUFFLE, SIZE_VARIABLE+1))
input_train_all_const[:, :-1] =  input_train_shuffle[:, :]
input_train_all_const[:, -1] = np.ones((SIZE_TRAIN_SHUFFLE))

pinv_input_train_all = np.linalg.pinv(input_train_all_const[:, :])
coeff_train_all = pinv_input_train_all[:, :] @ output_train_shuffle[:, :]
output_train_all_pred = input_train_all_const[:, :]@coeff_train_all[:, :]

input_test_all_const = np.zeros((SIZE_TEST, SIZE_VARIABLE+1))
input_test_all_const[:, :-1] =  input_test[:, :]
input_test_all_const[:, -1] = np.ones((SIZE_TEST))

output_test_all = input_test_all_const[:, :] @ coeff_train_all[:, :]
error_test_all = output_test[:, :] - output_test_all[:, :]
mse_test_all = np.average(error_test_all[:, :]**2, axis=0)

In [ ]:
print('mse_test_all:', np.average(mse_test_all))
print('mse_test_select:', np.average(mse_test_select))

plt.figure()
plt.title('Prediction of test data')

plt.ylabel('Prediction')
plt.xlabel('Time')

plt.plot(mse_test_all, label='ALL')
plt.plot(mse_test_select, label='Select')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim(0, DIM_OUTPUT-1)
plt.show()

In [ ]:
plt.figure()
plt.title('Prediction of train data')

plt.ylabel('Prediction')
plt.xlabel('Time')

idx = SIZE_TEST-1
plt.plot(output_train_all_pred[idx, :], label='ALL')
plt.plot(output_train_select_pred[idx, :], label='Select')
plt.plot(output_train_shuffle[idx, :], label='Data', color='black', linestyle='dashed')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim(0, DIM_OUTPUT-1)
plt.show()

In [ ]:
plt.figure()
plt.title('Prediction of test data')

plt.ylabel('Prediction')
plt.xlabel('Time')

idx = SIZE_TEST-1
plt.plot(output_test_all[idx, :], label='ALL')
plt.plot(output_test_select[idx, :], label='Select')
plt.plot(output_test[idx, :], label='Data', color='black', linestyle='dashed')
plt.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0)
plt.xlim(0, DIM_OUTPUT-1)
plt.show()

In [ ]:
np.save('model/esn_lorenz/mcmc.npy', mcmc)
np.save('model/esn_lorenz/state_IC_best_all.npy', state_IC_best_all)
np.save('model/esn_lorenz/state_select_best_all.npy', state_select_best_all)